# Process FFT Datasets Into Images

Here, we go through the classed functions for formatting the fft datasets into their image representations. We can:

- map data using the xyz coordinates using a PCA method
- map data using the inverse gain matrix transposed to map onto the parcellated 84 regions in MRI xyz space

These methods can be compared to determine the accuracy of encoding different types of structural information into the models.

In [1]:
import numpy as np
import os
import sys

sys.path.append('../')
from dnn.processing.format import formatfft

%load_ext autoreload
%autoreload 2

/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def main(fftdatadir, metadatadir, outputdatadir, gainmatfile):
    formatter = formatfft.FormatFFT(fftdatadir, metadatadir, outputdatadir)
    
    formatter.loadgainmat(gainmatfile)
    formatter.getdatafiles()
#     formatter.formatdata()




In [3]:
patid = 'id001_ac'

# the main data directories that there is raw, meta, and output data
metadatadir = '/Volumes/ADAM LI/pydata/metadata/'
rawdatadir = '/Volumes/ADAM LI/pydata/convertedtng/'
fftdatadir = '/Volumes/ADAM LI/pydata/output_fft/tng/win500_step250/'
outputdatadir = '/Volumes/ADAM LI/pydata/output_fft/asimages/regions/'

gainmatfile = os.path.join(metadatadir, patid, 'gain_inv-square.txt')
# run conversion
main(fftdatadir, metadatadir, outputdatadir, gainmatfile)

0


KeyError: 'metadata is not a file in the archive'